In [12]:
import pandas as pd
import numpy as np
import pickle
import re
import faiss
from sentence_transformers import SentenceTransformer
from pycaret.classification import load_model, predict_model
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Cargo el modelo de clasificación y vectorizador TF-IDF
modelo_clasificacion = load_model('modelo_Predicción_Category')

with open("tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

Transformation Pipeline and Model Successfully Loaded


In [15]:
# Cargo FAISS y modelo de embeddings
index = faiss.read_index("productos_faiss.index")  
modelo_embeddings = SentenceTransformer('all-mpnet-base-v2')

In [16]:
# Cargo el archivo correcto con embeddings
df_productos = pd.read_csv("productos_con_embeddings.csv")

In [17]:
# convierto id a string para evitar errores
df_productos['id'] = df_productos['id'].astype(str)

In [18]:
# Cargo los IDs originales de los productos en FAISS
df_ids = pd.read_csv("productos_ids.csv")
df_ids['id'] = df_ids['id'].astype(str)

In [19]:
#  creo una función pata predecir la categoría
def predecir_categoria(titulo):
    titulo_procesado = re.sub(r'[^a-zA-Z0-9 ]', '', titulo.lower())
    titulo_tfidf = vectorizer.transform([titulo_procesado]).toarray()
    df_titulo = pd.DataFrame(titulo_tfidf, columns=vectorizer.get_feature_names_out())
    pred = predict_model(modelo_clasificacion, data=df_titulo)
    categoria_predicha = pred.loc[0, 'prediction_label']
    
    if categoria_predicha not in df_productos['category_id'].unique():
        print("No se encontró una categoría válida.")
        return None
    
    print(f"Categoría predicha: {categoria_predicha}")
    return categoria_predicha

In [20]:
# Creo función para buscr productos similares en FAISS dentro de la categoría
def buscar_productos_similares(titulo_producto, top_n=50):
    # Predecir la categoría
    categoria_predicha = predecir_categoria(titulo_producto)
    if not categoria_predicha:
        return pd.DataFrame()  # Retorna vacío si no hay categoría

    # Filtrar productos de la categoría
    df_filtrado = df_productos[df_productos['category_id'] == categoria_predicha]
    if df_filtrado.empty:
        print("No hay productos en esta categoría.")
        return pd.DataFrame()

    # Obtener los embeddings de los productos en la categoría
    df_filtrado['title_embedding'] = df_filtrado['title_embedding'].apply(lambda x: np.array(eval(x)))
    embeddings_categoria = np.vstack(df_filtrado['title_embedding'].values)

    # Crear un índice FAISS SOLO para la categoría
    dimension = embeddings_categoria.shape[1]
    index_categoria = faiss.IndexFlatL2(dimension)
    index_categoria.add(embeddings_categoria)

    # Obtener el embedding del título ingresado
    query_embedding = modelo_embeddings.encode(titulo_producto).reshape(1, -1)

    # Buscar en FAISS SOLO dentro de la categori
    distances, indices = index_categoria.search(query_embedding, min(top_n, len(df_filtrado)))

    # Obtener los productos mas cercanos
    productos_encontrados = df_filtrado.iloc[indices[0]]

    # Seleccionar columnas relevantes
    productos_finales = productos_encontrados[['id', 'title', 'price', 'thumbnail', 'permalink']]

    return productos_finales

In [ ]:
# Prueba del sistema de recomendación
titulo_usuario = "cien años de soledad"
resultados = buscar_productos_similares(titulo_usuario)

✅ Categoría predicha: MCO1196


In [109]:
resultados

,id,title,price,thumbnail,permalink
22776,MCO2007476320,cien años de soledad (edicion ilustrada),-0.112121,http://http2.mlstatic.com/D_982658-MCO30276004...,https://www.mercadolibre.com.co/cien-anos-de-s...
22841,MCO1342668917,cien años de soledad - edicion conmemorativa,-0.112244,http://http2.mlstatic.com/D_858730-MLU78052698...,https://www.mercadolibre.com.co/cien-anos-de-s...
23420,MCO1529896679,"cien años de soledad t/d: , de gabriel garcía ...",-0.112295,http://http2.mlstatic.com/D_958330-MLA81567958...,https://www.mercadolibre.com.co/cien-anos-de-s...
23406,MCO2104660530,libro cien años de soledad: edición conmemorat...,-0.112264,http://http2.mlstatic.com/D_615487-MLA72665742...,https://www.mercadolibre.com.co/libro-cien-ano...
22898,MCO2348109730,"cien años de soledad, de gabriel garcía márque...",-0.112309,http://http2.mlstatic.com/D_939284-MLU75103866...,https://www.mercadolibre.com.co/cien-anos-de-s...
23086,MCO1499137738,deshacer la ansiedad,-0.112310,http://http2.mlstatic.com/D_766241-MCO49606072...,https://www.mercadolibre.com.co/deshacer-la-an...
23258,MCO1459631989,mil soles espléndidos ( libro y original ),-0.112346,http://http2.mlstatic.com/D_680457-MCO51025986...,https://www.mercadolibre.com.co/mil-soles-espl...
23699,MCO1150017841,la canción de aquiles,-0.112147,http://http2.mlstatic.com/D_805454-MCO49054558...,https://www.mercadolibre.com.co/la-cancion-de-...
23209,MCO1424685585,cien años de soledad ed. ilustrada gabriel gar...,-0.112112,http://http2.mlstatic.com/D_792524-MLU73141756...,https://www.mercadolibre.com.co/cien-anos-de-s...
23162,MCO2736814044,como hacer que te pasen cosas buenas nueva edi...,-0.112381,http://http2.mlstatic.com/D_952394-MLU73882725...,https://www.mercadolibre.com.co/como-hacer-que...
